In [ ]:
!pip install biopython scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from random import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
def load_sequences(file_paths, window=150, stride=50):
    sequences, labels = [], []
    for label, path in file_paths.items():
        for record in SeqIO.parse(path, "fasta"):
            seq = str(record.seq).replace("*", "")
            for i in range(0, len(seq) - window + 1, stride):
                frag = seq[i:i+window]
                if len(frag) == window:
                    sequences.append((frag, label))
    shuffle(sequences)
    fragments, labels = zip(*sequences)
    return list(fragments), list(labels)
def get_kmers(seq, k=3):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]
def build_kmer_features(sequences, k=3):
    all_kmers = set()
    kmer_counts = []
    for seq in sequences:
        kmers = get_kmers(seq, k)
        count = Counter(kmers)
        kmer_counts.append(count)
        all_kmers.update(count)
    all_kmers = sorted(list(all_kmers))
    df = pd.DataFrame(0, index=np.arange(len(sequences)), columns=all_kmers)
    for i, count in enumerate(kmer_counts):
        for kmer, freq in count.items():
            df.at[i, kmer] = freq
    return df
file_paths = {
    'Zika': '/content/drive/MyDrive/sagar/Zika_200.fasta',
    'Ebola': '/content/drive/MyDrive/sagar/Ebola_200.fasta',
    'SARS_CoV_2': '/content/drive/MyDrive/sagar/SARS_CoV_2_200.fasta',
    'Influenza_A': '/content/drive/MyDrive/sagar/Influenza_A_200.fasta',
    'Influenza_B': '/content/drive/MyDrive/sagar/Influenza_B_200.fasta',
    'Tuberculosis': '/content/drive/MyDrive/sagar/Tuberculosis_200.fasta',
    'Human': '/content/drive/MyDrive/sagar/Human_200.fasta'
}
data, labels = load_sequences(file_paths, window=150, stride=50)
print(f" Augmented sequences: {len(data)}")
X = build_kmer_features(data, k=3)
y_raw = pd.Series(labels)
y = pd.factorize(y_raw)[0]
num_classes = len(np.unique(y))
selector = SelectKBest(mutual_info_classif, k=200)
X_selected = selector.fit_transform(X, y)
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, stratify=y, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_cnn = np.expand_dims(X_train_scaled, axis=2)
X_test_cnn = np.expand_dims(X_test_scaled, axis=2)
y_train_cnn = to_categorical(y_train, num_classes)
y_test_cnn = to_categorical(y_test, num_classes)
model = Sequential([
    Conv1D(filters=24, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(48, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train_cnn, y_train_cnn,
    validation_split=0.1,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)
y_pred_probs = model.predict(X_test_cnn)
y_pred_cnn = np.argmax(y_pred_probs, axis=1)
acc_cnn = accuracy_score(y_test, y_pred_cnn)
print(f"\n Final CNN Accuracy: {acc_cnn * 100:.2f}%")
print("\n CNN Classification Report:\n")
print(classification_report(y_test, y_pred_cnn, target_names=np.unique(y_raw)))

 Augmented sequences: 9820
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


221/221 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6448 - loss: 1.0317 - val_accuracy: 0.9517 - val_loss: 0.2468
Epoch 2/50
221/221 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8844 - loss: 0.3603 - val_accuracy: 0.9669 - val_loss: 0.1617
Epoch 3/50
221/221 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9170 - loss: 0.2608 - val_accuracy: 0.9733 - val_loss: 0.1190
Epoch 4/50
221/221 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9337 - loss: 0.2105 - val_accuracy: 0.9746 - val_loss: 0.0993
Epoch 5/50
221/221 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9488 - loss: 0.1755 - val_accuracy: 0.9822 - val_loss: 0.0876
Epoch 6/50
221/221 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9497 - loss: 0.1514 - val_accuracy: 0.9835 - val_loss: 0.0775
Epoch 7/50
221/221 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9539 - loss: 0.1429 - val_accuracy: 0.9860 - val_loss: 0.0776
Epoch 8/50
221/221 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9580 - loss: 0.1305 - val_accuracy: 0.9860 - val